<a href="https://colab.research.google.com/github/ayushpradhananga/Deep-Learning-Projects/blob/main/Ayush_Pradhananga_MIS780A2_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Agricultural pest recognition with image data

## Table of Content

1. [Executive Summary](#cell_Summary)

2. [Data Preprocessing](#cell_Preprocessing)

3. [Predictive Modeling](#cell_model)

4. [Experiments Report](#cell_report)



<a id = "cell_Summary"></a>
## 1. Executive Summary

Problem Definition:
In this project the problem is classification of agricultural pests in images. Specifically, we have a dataset comprising 2,479 real images of five different types of agricultural pests: Ants, Bees, Grasshoppers, Moths, and Wasps. The goal is to build and evaluate machine learning models capable of accurately detecting and classifying these pests within images.


Proposed Approaches:
Two deep learning models with different architectures will be developed and compared for pest recognition. These models will involve various combinations of convolutional layers, pooling layers, fully connected layers, and dropout layers.


Major Findings:

Both the model was not reliable and performed poorly on the testing data. However, model 1 gave a better kappa. The models require alot of improvements such as hyper parameter tuning and improvement in model architechture. This is the limitation of this project.

Real-world Applicability: A robust model would have been applicable for agricultre sector. However, the models presented is not reliable for any business decision.

<a id = "cell_Preprocessing"></a>
## 2. Data Preprocessing

In [ ]:
%%html
<style>table {float:left}</style>
<style>img {float:left}</style>

####Import Libraries

In [ ]:
import numpy as np                   # Import NumPy for numerical operations.
import matplotlib.pyplot as plt      # Import Matplotlib for data visualization.
import random                        # Import random for randomization (used for various purposes).
from tensorflow import keras         # Import TensorFlow's Keras API.
import tensorflow as tf              # Import TensorFlow for low-level deep learning functionality.
import os                            # Import the 'os' module for operating system-related functions and utilities.

from sklearn.metrics import classification_report   # Import classification_report for generating classification performance metrics.
from sklearn.metrics import cohen_kappa_score       # Import cohen_kappa_score for calculating the Cohen's Kappa coefficient.
from sklearn.metrics import confusion_matrix        # Import confusion_matrix for creating confusion matrices.
from sklearn.metrics import ConfusionMatrixDisplay   # Import ConfusionMatrixDisplay for displaying confusion matrices.

from keras.datasets import cifar10       # Import the CIFAR-10 dataset, a popular image classification dataset.
from keras.models import Sequential    # Import Sequential for creating a sequential deep learning model.
from keras.layers.core import Dense, Dropout, Activation  # Import core components for building neural network layers.
from keras.utils import np_utils     # Import np_utils for handling data transformations.
from keras.preprocessing.image import ImageDataGenerator  # Import ImageDataGenerator for data augmentation.
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Flatten
# Import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, and Flatten for defining convolutional and pooling layers.
from keras.layers.normalization.batch_normalization import BatchNormalization
# Import BatchNormalization for normalizing layer activations.
from keras.callbacks import ModelCheckpoint, EarlyStopping  # Import ModelCheckpoint and EarlyStopping for model callbacks.
from keras.optimizers import SGD, RMSprop, Adam, Nadam  # Import various optimizers for training the model.
from keras.losses import categorical_crossentropy  # Import categorical_crossentropy as the loss function for multi-class classification.

# This section sets up your deep learning environment and imports the necessary libraries and modules.


ModuleNotFoundError: ignored

In [ ]:
#Check GPUs on this machine
tf.config.list_physical_devices('GPU')


Function to plot image

In [ ]:
#A custom function for plotting a grid of images
def plot_images(ims, figsize=(12,12), cols=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        if (ims.shape[-1] != 3):
            ims = ims = ims[:,:,:,0]
    f = plt.figure(figsize=figsize)
    rows=len(ims)//cols if len(ims) % cols == 0 else len(ims)//cols + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

Function to plot history

In [ ]:
def plot_hist(h, xsize=6, ysize=5):
    # Prepare plotting
    fig_size = plt.rcParams["figure.figsize"]
    plt.rcParams["figure.figsize"] = [xsize, ysize]

    # Get training and validation keys
    ks = list(h.keys())
    n2 = math.floor(len(ks)/2)
    train_keys = ks[0:n2]
    valid_keys = ks[n2:2*n2]

    # summarize history for different metrics
    for i in range(n2):
        plt.plot(h[train_keys[i]])
        plt.plot(h[valid_keys[i]])
        plt.title('Training vs Validation '+train_keys[i])
        plt.ylabel(train_keys[i])
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.draw()
        plt.show()

    return

###Mounting Google Drive

In [ ]:
from google.colab import drive

# Mount Google Drive for access.
drive.mount('/content/drive')

# List files in a specific directory.
!ls "/content/drive/My Drive/Colab Notebooks/Part2_agricultural_pests/"

###Import data

In [ ]:

# Set the paths to the folders containing the image files
ants_path = '/content/drive/MyDrive/Colab Notebooks/Part2_agricultural_pests/ants'
bees_path = '/content/drive/MyDrive/Colab Notebooks/Part2_agricultural_pests/bees'
grasshopper_path = '/content/drive/MyDrive/Colab Notebooks/Part2_agricultural_pests/grasshopper'
moth_path = '/content/drive/MyDrive/Colab Notebooks/Part2_agricultural_pests/moth'
wasp_path = '/content/drive/MyDrive/Colab Notebooks/Part2_agricultural_pests/wasp'
# get a list of all files in the folder
ants_file_list = os.listdir(ants_path)
bees_file_list = os.listdir(bees_path)
grasshopper_file_list = os.listdir(grasshopper_path)
moth_file_list = os.listdir(moth_path)
wasp_file_list = os.listdir(wasp_path)
# print the total number of files
print(f'Total number of files under ants folder are: {len(ants_file_list)}')
print(f'Total number of files under bees folder are: {len(bees_file_list)}')
print(f'Total number of files under grasshopper folder are: {len(grasshopper_file_list)}')
print(f'Total number of files under moth folder are: {len(moth_file_list)}')
print(f'Total number of files under wasp folder are: {len(wasp_file_list)}')

Read and decode raw image data from two folders then Assign labels based on the folder After that we Resize images to 100x100 resolution. Lastly, Store image data and labels in a list called 'data'.

In [ ]:
# Create a list to store the image data and labels
data = []

# Iterate through the files in the first folder
for file in os.listdir(ants_path):
  # Check if the file is a jpeg or jpg file
  if file.endswith('.jpeg') or file.endswith('.jpg'):
    # Load the image data from the file using TensorFlow
    img = tf.io.read_file(os.path.join(ants_path, file))
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, (100, 100))
    # Assign a label to the file
    label = 'Ants'
    # Add the image data and label to the data list
    data.append((img, label))

# Iterate through the files in the second folder
for file in os.listdir(bees_path):
  # Check if the file is a jpeg or jpg file
  if file.endswith('.jpeg') or file.endswith('.jpg'):
    # Load the image data from the file using TensorFlow
    img = tf.io.read_file(os.path.join(bees_path, file))
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, (100, 100))
    # Assign a label to the file
    label = 'Bees'
    # Add the image data and label to the data list
    data.append((img, label))

# Iterate through the files in the third folder
for file in os.listdir(grasshopper_path):
  # Check if the file is a jpeg or jpg file
  if file.endswith('.jpeg') or file.endswith('.jpg'):
    # Load the image data from the file using TensorFlow
    img = tf.io.read_file(os.path.join(grasshopper_path, file))
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, (100, 100))
    # Assign a label to the file
    label = 'Grasshopper'
    # Add the image data and label to the data list
    data.append((img, label))

# Iterate through the files in the fourth folder
for file in os.listdir(moth_path):
  # Check if the file is a jpeg or jpg file
  if file.endswith('.jpeg') or file.endswith('.jpg'):
    # Load the image data from the file using TensorFlow
    img = tf.io.read_file(os.path.join(moth_path, file))
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, (100, 100))
    # Assign a label to the file
    label = 'Moth'
    # Add the image data and label to the data list
    data.append((img, label))

 # Iterate through the files in the fifth folder
for file in os.listdir(wasp_path):
  # Check if the file is a jpeg or jpg file
  if file.endswith('.jpeg') or file.endswith('.jpg'):
    # Load the image data from the file using TensorFlow
    img = tf.io.read_file(os.path.join(wasp_path, file))
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, (100, 100))
    # Assign a label to the file
    label = 'Wasp'
    # Add the image data and label to the data list
    data.append((img, label))

Train- Test Split

In [ ]:
# Define the input shape based on image dimensions (100x100 pixels with 3 color channels).
input_shape = (100, 100, 3)

# Specify the number of classes:  5 classes).
num_classes = 5

# Split the shuffled data into training and testing sets (70% train, 30% test).
train_data, test_data = data[:int(len(data) * 0.7)], data[int(len(data) * 0.7):]


Training data consits of 70% and testing data consits of 30%.
Allocate X_train, X_test, Y_train, and Y_test.
Convert them into NumPy arrays for use in training a Convolutional Neural Network (CNN) module.


In [ ]:
# Extract the image data and labels from the training data
X_train, Y_train = zip(*train_data)

# Extract the image data and labels from the testing data
X_test, Y_test = zip(*test_data)

# Convert the image data and labels into NumPy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

# Reshape the training data
X_train = X_train.reshape(X_train.shape[0], *input_shape)

# Reshape the test data
X_test = X_test.reshape(X_test.shape[0], *input_shape)

###Data Normalization

In [ ]:
from tensorflow.keras.utils import to_categorical
# change integers to 32-bit floating point numbers
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalize each value for each pixel for the entire vector for each input
X_train /= 255
X_test /= 255

# print the shape of the reshaped data
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

In [ ]:
print('The original format of class of the first element in the training dataset is: ',Y_train[0], '\n')

import numpy as np
# Create a NumPy array with category strings
categories = np.array(['Ants', 'Bees', 'Grasshopper', 'Moth', 'Wasp'])

# Create a mapping from category strings to integers
category_map = {'Ants': 0, 'Bees': 1, 'Grasshopper': 2, 'Moth': 3, 'Wasp': 4}

# Encode the categories for both training and test sets
Y_train = np.array([category_map[category] for category in Y_train])
Y_test = np.array([category_map[category] for category in Y_test])

print('The unique integer mapping encoding format of the class of the first element in the training dataset is: ',Y_train[0])

# convert class vectors to binary class matrices
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)
# print the shape of the reshaped data
print("Training matrix shape", Y_train.shape)
print("Testing matrix shape", Y_test.shape)

Plotting image from the training data

In [ ]:
import matplotlib.pyplot as plt

# Set the default figure size
plt.rcParams['figure.figsize'] = (9, 9)

labels = ['Ants', 'Bees', 'Grasshopper', 'Moth', 'Wasp']

# Convert Y_train to integer labels if it's one-hot encoded
integer_labels = np.argmax(Y_train, axis=1)

# Create a 5x5 grid of subplots
fig, axes = plt.subplots(5, 5, figsize=(15, 15))

for i, ax in enumerate(axes.flat):
    # Display the image in the current subplot
    ax.imshow(X_train[i], cmap='gray')
    ax.set_title("{}".format(labels[integer_labels[i]]))
    ax.axis('off')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Show the figure
plt.show()

<a id = "cell_model"></a>
## 3. Predictive Modeling

###First Model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def model_1(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(4, 4), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    return model

EarlyStopping callback is used from Keras to stop the model training if the validation loss stops decreasing for a few epochs.

In [ ]:
# Keras callbacks (when Tensorboard installed)
keras_callbacks = [EarlyStopping(monitor='val_loss', patience=10, verbose=0)]

In [ ]:
model = model_1(input_shape, num_classes)

I've designed my model with a specific architecture in mind. It starts with a Convolutional Layer (Conv2D) with 32 filters and a 4x4 kernel size. This layer helps the model learn important features from the input images using the Rectified Linear Unit (ReLU) activation function.

After this Conv2D layer, I've added a MaxPooling Layer (MaxPooling2D) with a 2x2 pooling size. This layer reduces the spatial dimensions and retains crucial information from the previous layer.

Following the MaxPooling layer, I've incorporated a Flatten Layer. This step is crucial as it transforms the 2D feature maps into a 1D vector, making it suitable for further processing in the dense layers.

The Fully Connected Layers (Dense) are next in line. The first Dense layer consists of 50 neurons, employing the ReLU activation function. Right after this layer, I've included a Dropout layer with a dropout rate of 0.25. Dropout helps prevent overfitting by randomly deactivating neurons during training.

Lastly, there's the final Dense layer, which has a number of neurons equal to num_classes. This layer utilizes the softmax activation function, which is particularly useful for multi-class classification tasks. It provides class probabilities as its output, enabling the model to make predictions based on these probabilities.

In [ ]:
# Compile the model with the Adam optimizer
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
              metrics='accuracy')

In [ ]:
hist = model.fit(X_train, Y_train,
      batch_size=128,
      epochs=50,
      verbose=2,
      validation_data=(X_test, Y_test),
      validation_split=0.3,
      callbacks=keras_callbacks)

In [ ]:
# Evaluate on training data
train_score = model.evaluate(X_train, Y_train, verbose=0)
print('Train loss:', (train_score[0], 4))
print('Train accuracy:', (train_score[1], 4), '\n')

# Evaluate on test data
test_score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', (test_score[0], 4))
print('Test accuracy:', (test_score[1], 4))

In [ ]:
# Make predictions on the test data
y_pred = model.predict(X_test)
# Convert y_pred to categorical labels
predicted_labels = np.argmax(y_pred, axis=1)

# Convert Y_test to categorical labels
true_labels = np.argmax(Y_test, axis=1)
from sklearn.metrics import classification_report

# Calculate the kappa score
kappa = cohen_kappa_score(true_labels, predicted_labels)
print("The result of Kappa is:", round(kappa, 3))

# Generate the classification report
report = classification_report(true_labels, predicted_labels, target_names=labels)
print(report)


In [ ]:
import pandas as pd
import math
plot_hist(pd.DataFrame(hist.history))

###Second Model

In [ ]:
from keras.layers import BatchNormalization, Dense, Dropout
from keras.models import Sequential
from keras.regularizers import l2  # Import L2 regularization

def model_2():
    model = Sequential()
    model.add(Conv2D(10, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    return model

In [ ]:
complexmodel = model_2()

In [ ]:
# Compile the model with the RMSprop optimizer
complexmodel.compile(loss=categorical_crossentropy,
              optimizer= RMSprop(lr=0.001,decay=1e-6),
              metrics='accuracy')

In [ ]:
history = complexmodel.fit(X_train, Y_train,
      batch_size=128,
      epochs=100,
      verbose=2,
      validation_data=(X_test, Y_test),
      validation_split=0.3,
      callbacks=keras_callbacks)

In [ ]:
# Evaluate on training data
train_score1 = complexmodel.evaluate(X_train, Y_train, verbose=0)
print('Train loss:', (train_score1[0], 4))
print('Train accuracy:', (train_score1[1], 4), '\n')

# Evaluate on test data
test_score1 = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', (test_score1[0], 4))
print('Test accuracy:', (test_score1[1], 4))

In [ ]:
# Make predictions on the test data
y_pred1 = complexmodel.predict(X_test)
# Convert y_pred to categorical labels
predicted_labels1 = np.argmax(y_pred1, axis=1)

# Convert Y_test to categorical labels
true_labels1 = np.argmax(Y_test, axis=1)
from sklearn.metrics import classification_report

# Calculate the kappa score
kappa1 = cohen_kappa_score(true_labels1, predicted_labels1)
print("The result of Kappa is:", round(kappa1, 3))

In [ ]:
plot_hist(pd.DataFrame(history.history))

<a id = "cell_report"></a>
## 4. Experiments Report

The first model performed better than the second model, although both the models are not good. The first model got an accuracy of 21.8% and kappa of 15.1% on testing data. The second model on the other hand got the same accuracy but a lower kappa of 13%.

The models could be improved with changing the architecture and hyperparameters. However, upon experimenting, the more complex the model I built, the worse was the performance. The model is not reliable and can not be used for a real business decision. The model needs to be improved before using it for any business purpose.